In [1]:
import pandas as pd
import numpy as np
import os

# Read in data

Gold standard data

In [2]:
gs_dir = "../data/ASM/gold_standard_april_25_2018/"
cols = ["subject_id", "frame", "line", "transcription"]
gold_standard = pd.DataFrame(columns=cols)
gs_subj = [f.replace(".txt", "") for f in os.listdir(gs_dir)]
for f in os.listdir(gs_dir):
    text = open(os.path.join(gs_dir, f)).read()
    subject = f.replace(".txt", "")

    # text = trans[0]
    text_pages = text.split("\n\n")

    for i in range(len(text_pages)):
        text_lines = text_pages[i].split("\n")
        for j in range(len(text_lines)):
            line = text_lines[j][:-8]
            new_trans = {"subject_id":[subject],
                         "frame":[i],
                         "line":[j],
                         "transcription":[line]}

            new_trans = pd.DataFrame.from_dict(new_trans)
            gold_standard = gold_standard.append(new_trans)

gold_standard["frame"] = gold_standard["frame"].apply(int)
gold_standard["line"] = gold_standard["line"].apply(int)

In [3]:
full_data = pd.read_csv("../data/ASM/full_train2.csv", sep="\t", encoding="ISO-8859-1")

# Get character error rates

In [4]:
# get character error rate
def cer(r, h):
    r = list(r)
    h = list(h)
    # initialization
    d = np.zeros((len(r)+1)*(len(h)+1), dtype=np.uint8)
    d = d.reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)

    return d[len(r)][len(h)]/(1.0*len(r))

In [5]:
cols = ["subject_id", "frame", "line", "gs", "volunteer", "cer"]
match_data = pd.DataFrame(columns=cols)

for i in range(len(full_data)):
    if i % 200 == 0:
        print(i, end="\t", flush=True)
    cur_trans = full_data.iloc[i]
    if type(cur_trans.transcription) == str:
        subj = str(cur_trans["subject_id"])
        frame = cur_trans["frame"]
        line = cur_trans["j"]
        subj_match = gold_standard["subject_id"] == subj
        fram_match = gold_standard["frame"] == frame
        line_match = gold_standard["line"] == line
        match = np.logical_and(np.logical_and(subj_match, fram_match), line_match)
        if sum(match) > 0:
            cur_gs = gold_standard[match].iloc[0]
            gst = cur_gs.transcription
            vot = cur_trans.transcription
            if abs(len(gst) - len(vot)) <= 6:
                cerrate = cer(gst, vot)
                if cerrate < 0.5:
                    new_trans = {"subject_id":[subj],
                                 "frame":[frame],
                                 "line":[line],
                                 "gs":[gst],
                                 "volunteer":[vot],
                                 "cer":cerrate}
                    new_trans = pd.DataFrame.from_dict(new_trans)
                    match_data = match_data.append(new_trans)


0	200	400	600	800	1000	1200	1400	1600	1800	2000	2200	2400	2600	2800	

In [6]:
match_data.to_csv("data/volunteer_vs_gs.csv", sep="\t", index=False)

# Not used, maybe useful

Volunteers

In [ ]:
full_data = pd.read_csv("../data/ASM/full_train.csv", sep="\t")

def in_gs(subject_id):
    return str(subject_id) in gs_subj
full_data["gold_standard"] = full_data["subject_id"].apply(in_gs)

full_data = full_data[full_data["gold_standard"]]

Online training

In [ ]:
online_data = pd.read_csv("data/online_training/metrics_image89000.csv")

def get_line_info(filename, part):
    filename = filename.replace("/home/danny_hanson52/text_recognition/data/ASM/Images/", "")
    filename = filename.replace(".png", "")
    return filename.split("_")[part]

online_data["subject_id"] = online_data["filenames"].apply(get_line_info, part=0)
online_data["classification_id"] = online_data["filenames"].apply(get_line_info, part=1)
online_data["frame"] = online_data["filenames"].apply(get_line_info, part=2)
online_data["line"] = online_data["filenames"].apply(get_line_info, part=3)

def in_gs(subject_id):
    return subject_id in gs_subj
online_data["gold_standard"] = online_data["subject_id"].apply(in_gs)

online_gs = online_data[online_data["gold_standard"]]
online_gs = online_gs[online_gs["pred"] == "pred"]